In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import sklearn.decomposition
import matplotlib.animation as animation
import openmm.unit as u
import deeptime as dt
import networkx as nx


from cycler import cycler
plt.rcParams['font.size'] = 7
plt.rcParams['font.family'] = 'Calibri'
plt.rcParams['mathtext.default'] = 'regular'

plt.rcParams['axes.labelpad'] = 0.1
plt.rcParams['axes.labelsize'] = 7
plt.rcParams['axes.linewidth'] = 0.7
colorList = ['#000000', '#FF0000', '#0000FF', '#FF64FF',
             '#2192E5', '#009900', '#FF8700', '#F2E100', 
             '#FFDE3B', '#5A1AF5']
plt.rcParams['axes.prop_cycle'] = cycler(color=colorList)
plt.rcParams['axes.spines.right'] = True
plt.rcParams['axes.spines.top'] = True
plt.rcParams['axes.titlepad'] = 0.1
plt.rcParams['axes.titlesize'] = 7*1.2

plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = (3.25, 2.5)
plt.rcParams['figure.labelsize'] = 7
plt.rcParams['figure.titlesize'] = 7*1.2

plt.rcParams['legend.fancybox'] = False
plt.rcParams['legend.fontsize'] = 7
plt.rcParams['legend.frameon'] = False
plt.rcParams['legend.markerscale'] = 1
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.title_fontsize'] = 7*1.2

plt.rcParams['lines.dash_capstyle'] = 'round'
plt.rcParams['lines.dash_joinstyle'] = 'round'
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.markersize'] = 1

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.transparent'] = True
plt.rcParams['savefig.bbox'] = 'tight'

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['xtick.labelsize'] = 7
plt.rcParams['xtick.major.size'] = 3
# plt.rcParams['xtick.major.top'] = True
plt.rcParams['xtick.major.width'] = 0.6
plt.rcParams['xtick.minor.size'] = 1.5
plt.rcParams['xtick.minor.visible'] = True
# plt.rcParams['xtick.minor.top'] = True
plt.rcParams['xtick.minor.width'] = 0.6
plt.rcParams['xtick.top'] = True

plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['ytick.labelsize'] = 7
plt.rcParams['ytick.major.size'] = 3
# plt.rcParams['ytick.major.right'] = True
plt.rcParams['ytick.major.width'] = 0.6
plt.rcParams['ytick.minor.size'] = 1.5
# plt.rcParams['ytick.minor.right'] = True
plt.rcParams['ytick.minor.width'] = 0.6
plt.rcParams['ytick.right'] = True
plt.rcParams['ytick.minor.visible'] = True

# Utils

In [2]:
def SVD(rateMatrix):
    """ Utility function for SVD of Q matrix
        returns the Eigenvalues and Eigenvectors of the Rate Matrix Q
    """
    # it is transpose because the function returns 
    # the right eigenvec's but we want left eigenvectors
    eig_val, eig_vec = np.linalg.eig(rateMatrix.T)
    return (eig_val, eig_vec)

def Prob_t(uniqueStates, val, vec, time):
    """ returns the time evolution of the probabilites
        Note that this can be used for simulating the time evolution of the probabilities and
        as the starting point for the construction of a Transition Matrix at the
        desired lagtime
    """
    # this works because of how numpy casts when performing the operation
    D = np.eye(uniqueStates) * np.exp(val * time)
    return np.linalg.inv(vec.T) @ D @ vec.T

def MSMtraj(steps, transitionMatrix):
    """ returns the MSM and a discrete time discrete state 
        trajectory from the Transition Matrix
        at the spicified lagtime of shape (steps,)
    """
    MSM_true = dt.markov.msm.MarkovStateModel(transitionMatrix)
    traj_discretized = MSM_true.simulate(steps)
    return (MSM_true, traj_discretized)

In [3]:
def isallowedTransition(i, j):
    """
    Function for determining if i -> j transition is allowed.
    """
    diff = j - i
    numForward = np.sum(diff == 1)
    numBackward = np.sum(diff == -1)
    if numForward == 1 and numBackward == 1:
        return 1
    elif (numForward == 1 or numBackward == 1) and (diff[0] == 1): #or diff[-1] == 1): # not consider because assymmetric
        return 1
    return 0

# ASEP Model

We use 8 sites, $\alpha=\beta=p=1$, and $q=\frac{1}{3}$

In [4]:
from itertools import combinations, permutations, combinations_with_replacement, product

In [5]:
microstates = np.array(list(product([0,1], repeat=8)))

In [6]:
microstates

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]], shape=(256, 8))

In [7]:
microstates.shape, 2**8

((256, 8), 256)

In [8]:
microstates[0], microstates[2]

(array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 1, 0]))

In [22]:
[1, -1] in (microstates[2] - microstates[1])

ValueError: operands could not be broadcast together with shapes (8,) (2,) 

In [20]:
i = 1
for j in range(microstates.shape[0]):
    if isallowedTransition(microstates[i], microstates[j]):
        print(i, j, microstates[i], microstates[j],
              microstates[j] - microstates[i])

1 2 [0 0 0 0 0 0 0 1] [0 0 0 0 0 0 1 0] [ 0  0  0  0  0  0  1 -1]
1 4 [0 0 0 0 0 0 0 1] [0 0 0 0 0 1 0 0] [ 0  0  0  0  0  1  0 -1]
1 8 [0 0 0 0 0 0 0 1] [0 0 0 0 1 0 0 0] [ 0  0  0  0  1  0  0 -1]
1 16 [0 0 0 0 0 0 0 1] [0 0 0 1 0 0 0 0] [ 0  0  0  1  0  0  0 -1]
1 32 [0 0 0 0 0 0 0 1] [0 0 1 0 0 0 0 0] [ 0  0  1  0  0  0  0 -1]
1 64 [0 0 0 0 0 0 0 1] [0 1 0 0 0 0 0 0] [ 0  1  0  0  0  0  0 -1]
1 128 [0 0 0 0 0 0 0 1] [1 0 0 0 0 0 0 0] [ 1  0  0  0  0  0  0 -1]
1 129 [0 0 0 0 0 0 0 1] [1 0 0 0 0 0 0 1] [1 0 0 0 0 0 0 0]
1 130 [0 0 0 0 0 0 0 1] [1 0 0 0 0 0 1 0] [ 1  0  0  0  0  0  1 -1]
1 132 [0 0 0 0 0 0 0 1] [1 0 0 0 0 1 0 0] [ 1  0  0  0  0  1  0 -1]
1 134 [0 0 0 0 0 0 0 1] [1 0 0 0 0 1 1 0] [ 1  0  0  0  0  1  1 -1]
1 136 [0 0 0 0 0 0 0 1] [1 0 0 0 1 0 0 0] [ 1  0  0  0  1  0  0 -1]
1 138 [0 0 0 0 0 0 0 1] [1 0 0 0 1 0 1 0] [ 1  0  0  0  1  0  1 -1]
1 140 [0 0 0 0 0 0 0 1] [1 0 0 0 1 1 0 0] [ 1  0  0  0  1  1  0 -1]
1 142 [0 0 0 0 0 0 0 1] [1 0 0 0 1 1 1 0] [ 1  0  0  0  1  1  1 -

In [ ]:
Q = 